In [2]:
# # load dotenv + imports for retriever tool
from getpass import getpass
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma 
from langchain.tools import tool
import langchainhub as hub
import chromadb
from langchain_google_genai import GoogleGenerativeAIEmbeddings


api_key = os.getenv("CHROMA_API_KEY") or getpass("Paste CHROMA_API_KEY: ")
tenant = os.getenv("CHROMA_TENANT") or input("Enter CHROMA_TENANT: ")
database = os.getenv("CHROMA_DATABASE") or input("Enter CHROMA_DATABASE: ")
gemini_key = os.getenv("GEMINI_API_KEY") or getpass("Enter GEMINI_API_KEY: ")

# Load environment variables from chromaDB --CloudClient
client = chromadb.CloudClient(
    api_key=api_key,
    tenant=tenant,
    database=database
)
collection = client.get_collection(name="nice_org")

# load env to variable for langchain
os.environ["GOOGLE_API_KEY"] = gemini_key
# connecting embeddings with Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

vectorstore = Chroma(
    client=client,
    collection_name="nice_org",
    embedding_function=embeddings
)

print("Connection successful")

Connection successful
